In [1]:
import pandas as pd
import numpy as np

cols = set(np.arange(44))
irrelevant = set([0, 1, 2, 7, 8, 9, 10, 17, 22, 23, 24, 25, 26, 27, 28, 
                  29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43])
useful_cols = list(cols.difference(irrelevant))
df = pd.read_excel('menustat_2021_dataset.xlsx', usecols=useful_cols)

In [2]:
df = df[~(df['limited_time_offer'] == 1)]
df.drop('limited_time_offer', axis=1, inplace=True)
df[~(df['restaurant'] == 'Chick-Fil-A Lemonade, Medium')]
df = df[df['item_description'].str.lower().str.contains("toppings|topping|ingredient|available in|choice in|sides|condiments|sides|own")==False]

In [3]:
import requests
from bs4 import SoupStrainer, BeautifulSoup as bs
from bs4 import SoupStrainer
import re

states = ['new york', 'california', 'georgia', 'texas', 'minnesota', 'missouri']

def get_restaurant_links(desc, loc="nyc"):
    strainer = SoupStrainer("a", attrs={"href": True})
    url = f'https://www.yelp.com/search?find_desc={desc}&find_loc={loc}&sortby=review_count'
    page = requests.get(url)
    soup = bs(page.content, "html.parser", parse_only=strainer)
    links = [link.get('href') for link in soup]
    url_desc = desc.lower().split(' ')[0].replace("'", '')
    regex = f'/biz/{url_desc}.*'
    filtered = filter(lambda link : re.match(regex, link) and not 
    re.match(r'.*?hrid.*', link), links)
    full_links = map(lambda link : f"https://www.yelp.com{link}", filtered)
    return set(full_links)

full_reviews = []
restaurants = df['restaurant'].unique()

# preliminary testin
for restaurant in restaurants[:2]:
    reviews = set()
    for url in get_restaurant_links(restaurant):
        if len(reviews) > 30:
            break
        strainer = SoupStrainer("ul")
        soup = bs(requests.get(url).content, "html.parser", parse_only=strainer)
        for div in soup.find_all('div'):
            rating = div.find('div', attrs = {'role': 'img', 'aria-label': True})
            para = div.find('p')
            if para and rating and para.span:
                rating = rating['aria-label']
                rating = int(rating[:rating.find(' ')])
                reviews.add((rating, para.text))
    full_reviews.append((restaurant, reviews))
full_reviews

[('7-Eleven',
  {(1,
    '$2.06 for a small cup of coffee??..wtf and it was warm and the dude/manager had an attitude and pointed to a corner where a microwave was and said "press 2"....how about you tke responsibility and and make a new pot!... terrible!'),
   (1,
    "*STAFF PHYSICALLY INTIMIDATING CUSTOMERS*I came in to the store to return bottles to redeem my 5c deposit for each. The associate behind the counter was very adamant that he is *not* going to accept it. When I asked why, he said 'because I am not going to accept it'. I told him he, and the store, is obliged to accept it according to the New York State Returnable Container Act. I told him that the store charges 5c for each container, and I have the right to get it back. He was absolutely rude about the return, tried to physically intimidate me by telling me to come closer to him, and raising his voice. At the very end, he told me he is not doing this because he's been working there for 5 years and no one ever asked for i

In [4]:
def add_quote(string):
    return '"' + string.strip() + '"'
for text_field in ['food_category', 'restaurant', 'item_name', 'item_description']:
    df[text_field] = df[text_field][~df[text_field].isna()].astype(str).map(add_quote)
df.to_csv('fastfood.csv', index=False, encoding='utf-8') 